<a href="https://colab.research.google.com/github/CryptoManiacMS/colab/blob/main/flux-img-generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 STEP 1

In [1]:
# ✅ Remove qualquer versão antiga ou corrompida do repositório
!rm -rf /content/TotoroUI

# ✅ Remove possíveis restos bugados do torch
!rm -rf /usr/local/lib/python3.11/dist-packages/~orch*

# ✅ Limpa cache do pip (boa prática no Colab)
!pip cache purge

Files removed: 0


## 🚀 STEP 2

### Reinicie o ambiente de execução neste ponto!
### > Menu: Ambiente de execução > Reiniciar ambiente de execução
### ✅ Depois que o ambiente reiniciar, volte aqui e rode tudo abaixo:

In [ ]:
# ✅ Clona o repositório limpo
%cd /content
!git clone -b totoro3 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

# ✅ Instala as dependências corretas (versões compatíveis)
!pip install -q torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 \
  torchsde einops diffusers accelerate xformers==0.0.28.post2

# ✅ Instala o aria2 para fazer downloads rápidos dos modelos
!apt -y install -qq aria2

# ✅ Baixa os modelos necessários
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8.safetensors \
  -d /content/TotoroUI/models/unet -o flux1-dev-fp8.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft \
  -d /content/TotoroUI/models/vae -o ae.sft

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors \
  -d /content/TotoroUI/models/clip -o clip_l.safetensors

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp8_e4m3fn.safetensors \
  -d /content/TotoroUI/models/clip -o t5xxl_fp8_e4m3fn.safetensors


# 🚀 STEP 3

In [3]:
# Vai para a pasta do projeto
%cd /content/TotoroUI

# Importa os módulos necessários
import random
import torch
import numpy as np
from PIL import Image
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro import model_management

# Prepara os loaders
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

# Carrega os modelos
with torch.inference_mode():
    clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    unet = UNETLoader.load_unet("flux1-dev-fp8.safetensors", "fp8_e4m3fn")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

# Função auxiliar pra garantir tamanho múltiplo de 16
def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    n2 = m * (q + 1) if (n * m) > 0 else m * (q - 1)
    return n1 if abs(n - n1) < abs(n - n2) else n2

/content/TotoroUI


OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 12.12 MiB is free. Process 160433 has 14.73 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 102.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# 🚀 STEP 4

## Configurações:

- positive_prompt: Aqui você insere o prompt desejado
- width, height: Dimensões da imagem a ser gerada
- steps: Quantidade de etapas de refinamento
  - 10~20 steps → rápido, rascunho ou testes.
  - 30~50 steps → boa qualidade geral.
  - 60~100+ steps → alta qualidade, mas demora mais.

- sampler_name:

| Sampler       | Estilo                           | Detalhe                                    |
| ------------- | -------------------------------- | ------------------------------------------ |
| `euler`       | Limpo, traços nítidos            | Rápido e previsível                        |
| `euler_a`     | Artístico, mais "caótico"        | Pode gerar linhas e texturas interessantes |
| `ddim`        | Suave, sem muito ruído           | Mais rápido, às vezes menos detalhado      |
| `dpm2`        | Mais detalhado, traço firme      | Requer mais steps pra brilhar              |
| `dpm2_a`      | Tipo `euler_a`, mas mais preciso | Detalhes ricos, mas pode ter artefato      |
| `dpm++2m`     | Alta qualidade, clean            | Um dos mais equilibrados pra realismo      |
| `dpm++2m_sde` | Alta qualidade, mais aleatório   | Melhor em variações criativas              |
| `lms`         | Linhas suaves, bom equilíbrio    | Costuma funcionar bem em prompts complexos |
| `heun`        | Similar ao euler                 | Outra opção rápida e estável               |

- scheduler

| Scheduler     | Como age                                             |
| ------------- | ---------------------------------------------------- |
| `simple`      | Direto, funciona bem no geral                        |
| `exponential` | Remove ruído mais devagar no começo, acelera no fim  |
| `linear`      | Remove o ruído num ritmo constante (mais previsível) |
| `cosine`      | Começa rápido, suaviza no fim                        |
| `polynomial`  | Personalizado, dependendo do grau que você definir   |
| `sigmoid`     | Começa suave, acelera no meio, suaviza no fim        |

<br><br>
# 💡Dicas de uso
`euler + simple`: seguro, rápido, bom pra começar.

`dpm++2m + cosine`: qualidade top, mais steps, imagem mais limpa.

`euler_a + exponential`: gera variações criativas, mais caótico/artístico.

In [2]:
# Gera a imagem
with torch.inference_mode():
    positive_prompt = "black forest toast spelling out the words 'FLUX DEV', tasty, food photography, dynamic shot"
    width = 1024
    height = 1024
    seed = random.randint(0, 18446744073709551615)
    steps = 20
    sampler_name = "euler"
    scheduler = "simple"

    print(f"Seed usado: {seed}")

    cond, pooled = clip.encode_from_tokens(clip.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]

    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)

    model_management.soft_empty_cache()

    decoded = VAEDecode.decode(vae, sample)[0].detach()

    # Salva a imagem
    Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0]).save("/content/flux.png")

    # Exibe no notebook
    Image.fromarray(np.array(decoded * 255, dtype=np.uint8)[0])

10454769290812543217


NameError: name 'clip' is not defined